# Install & import libraries

In [1]:
%pip install -q chess stockfish python-telegram-bot==13 cairosvg zstandard

Note: you may need to restart the kernel to use updated packages.


# Introduction to libraries used (with examples)

## Python Chess

python-chess is a chess library for Python, with move generation, move validation, and support for common formats. Read the documentation here: https://python-chess.readthedocs.io/en/latest/

In [ ]:
import chess, chess.svg
FEN = "r6k/pp2r2p/4Rp1Q/3p4/8/1N1P2R1/PqP2bPP/7K b - - 0 24"
Moves = "f2g3 e6e7 b2b1 b3c1 b1c1 h6c1"
Moves = Moves.split(" ")

In [ ]:
# Load FEN position
board = chess.Board(FEN)
# Display board
display(board)

In [ ]:
# Make moves
move_uci = Moves[0]
move_san = board.san(chess.Move.from_uci(move_uci))

# Make the move
board.push(chess.Move.from_uci(move_uci))

# Display
print("Move in Universal Chess Interface (UCI): {}".format(move_uci))
print("Move in Standard Algebraic Notation (SAN): {}".format(move_san))
display(board)

### Exporting SVG image and convert it to PNG format

In [ ]:
from cairosvg import svg2png
import chess.svg

boardsvg = chess.svg.board(board=board)
svg2png(bytestring=boardsvg,write_to='output.png')

## Stockfish

Stockfish is a chess engine used for evaluation positions and generating best moves. Read the python stockfish documentation here: https://pypi.org/project/stockfish/

In [4]:
import requests
from stockfish import Stockfish

# Download stockfish from github
stockfishURL = "https://github.com/official-stockfish/Stockfish/releases/download/sf_16/stockfish-ubuntu-x86-64-avx2.tar"
response = requests.get(stockfishURL)

with open("stockfish.tar", "wb") as f:
    f.write(response.content)

# Uncompress the tar file
!tar -xvf stockfish.tar

stockfish/
stockfish/CITATION.cff
stockfish/stockfish-ubuntu-x86-64-avx2
stockfish/src/
stockfish/src/benchmark.cpp
stockfish/src/uci.cpp
stockfish/src/types.h
stockfish/src/position.cpp
stockfish/src/position.h
stockfish/src/tune.cpp
stockfish/src/benchmark.h
stockfish/src/search.cpp
stockfish/src/endgame.cpp
stockfish/src/thread.h
stockfish/src/nnue/
stockfish/src/nnue/evaluate_nnue.h
stockfish/src/nnue/nnue_common.h
stockfish/src/nnue/layers/
stockfish/src/nnue/layers/clipped_relu.h
stockfish/src/nnue/layers/simd.h
stockfish/src/nnue/layers/affine_transform_sparse_input.h
stockfish/src/nnue/layers/affine_transform.h
stockfish/src/nnue/layers/sqr_clipped_relu.h
stockfish/src/nnue/features/
stockfish/src/nnue/features/half_ka_v2_hm.h
stockfish/src/nnue/features/half_ka_v2_hm.cpp
stockfish/src/nnue/nnue_architecture.h
stockfish/src/nnue/evaluate_nnue.cpp
stockfish/src/nnue/nnue_feature_transformer.h
stockfish/src/nnue/nnue_accumulator.h
stockfish/src/incbin/
stockfish/src/incbin/incbin

In [ ]:
# Initialize stockfish from the downloaded path
stockfish = Stockfish(path="/content/stockfish/stockfish-ubuntu-x86-64-avx2")

# Set stockfish configurations
stockfish.set_depth(15)
stockfish.set_elo_rating(2000)

In [ ]:
FEN = "r6k/pp2r2p/4Rp1Q/3p4/8/1N1P2R1/PqP2bPP/7K b - - 0 24"
# Set to starting position
stockfish.set_fen_position(FEN)

# Instruct stockfish to generate k top moves
k = 10
top_moves = stockfish.get_top_moves(k)
print(top_moves)

[{'Move': 'b2b1', 'Centipawn': -441, 'Mate': None}, {'Move': 'f2e3', 'Centipawn': -208, 'Mate': None}, {'Move': 'e7f7', 'Centipawn': -17, 'Mate': None}, {'Move': 'e7c7', 'Centipawn': 27, 'Mate': None}, {'Move': 'e7d7', 'Centipawn': 121, 'Mate': None}, {'Move': 'f2g3', 'Centipawn': 606, 'Mate': None}, {'Move': 'f2c5', 'Centipawn': 659, 'Mate': None}, {'Move': 'a8g8', 'Centipawn': 943, 'Mate': None}, {'Move': 'b2c1', 'Centipawn': None, 'Mate': 8}, {'Move': 'a7a5', 'Centipawn': None, 'Mate': 8}]


In [ ]:
best_move = top_moves[0]["Move"]
print(best_move)

b2b1


## Lichess puzzle dataset

Lichess provides an open database with chess puzzles here: https://database.lichess.org/?ref=propelauth.com#puzzles . For the telegram bot, we download and filter the puzzles we want.

In [5]:
import requests, zstandard
import pandas as pd

# Download zst file
puzzleURL = "https://database.lichess.org/lichess_db_puzzle.csv.zst"
response = requests.get(puzzleURL)

with open("lichess_db_puzzle.csv.zst", "wb") as f:
    f.write(response.content)


# Uncompress zst file
with open("lichess_db_puzzle.csv.zst", "rb") as f:
    decomp = zstandard.ZstdDecompressor()
    with open("lichess_db_puzzle.csv", 'wb') as destination:
        decomp.copy_stream(f, destination)

In [6]:
df = pd.read_csv("lichess_db_puzzle.csv")
print(df.shape)
df.head()

(3466049, 10)


,PuzzleId,FEN,Moves,Rating,RatingDeviation,Popularity,NbPlays,Themes,GameUrl,OpeningTags
0,00008,r6k/pp2r2p/4Rp1Q/3p4/8/1N1P2R1/PqP2bPP/7K b - ...,f2g3 e6e7 b2b1 b3c1 b1c1 h6c1,1758,75,94,4412,crushing hangingPiece long middlegame,https://lichess.org/787zsVup/black#48,NaN
1,0000D,5rk1/1p3ppp/pq3b2/8/8/1P1Q1N2/P4PPP/3R2K1 w - ...,d3d6 f8d8 d6d8 f6d8,1511,74,96,23054,advantage endgame short,https://lichess.org/F8M8OS71#53,NaN
2,0008Q,8/4R3/1p2P3/p4r2/P6p/1P3Pk1/4K3/8 w - - 1 64,e7f7 f5e5 e2f1 e5e6,1292,75,100,116,advantage endgame rookEndgame short,https://lichess.org/MQSyb3KW#127,NaN
3,0009B,r2qr1k1/b1p2ppp/pp4n1/P1P1p3/4P1n1/B2P2Pb/3NBP...,b6c5 e2g4 h3g4 d1g4,1088,74,86,547,advantage middlegame short,https://lichess.org/4MWQCxQ6/black#32,Kings_Pawn_Game Kings_Pawn_Game_Leonardis_Vari...
4,000Vc,8/8/4k1p1/2KpP2p/5PP1/8/8/8 w - - 0 53,g4h5 g6h5 f4f5 e6e5 f5f6 e5f6,1556,81,89,81,crushing endgame long pawnEndgame,https://lichess.org/l6AejDMO#105,NaN


File size is too large for PythonAnywhere server, remove rows/columns to reduce < 100 mb

In [7]:
df2 = df[df["Themes"].str.contains("short")]
df2 = df2[df2["Rating"] > 1350]

# Grab the first 1 million rows
df2 = df2.iloc[:1000000]

# Keep only FEN, moves and rating rows
df2 = df2[["FEN","Moves","Rating"]]
df2.head()

# Write cleaned csv file
df2.to_csv("chess_puzzles.csv")

In [50]:
import pandas as pd
rating = 1500
rating_upper = rating+50
rating_lower = rating-50
filtered_df = df[(df.Rating>=rating_lower) & (df.Rating<=rating_upper)]
row = df.sample()
df.max()["Rating"]
df.min()["Rating"]

1351

# Telegram Bot

For the telegram API, we use the python-telegram-bot **v13 (not v20)** library. Read the documentation here: https://github.com/python-telegram-bot/python-telegram-bot/blob/v13.x . Run the cells below and the bot should work (@NTU_chess_test_bot)

## Chess & Utility functions

## Telegram functions

In [12]:
import chess, chess.svg, random, datetime, logging
from stockfish import Stockfish
from cairosvg import svg2png
import pandas as pd
from telegram import (
    Poll,
    KeyboardButton,
    KeyboardButtonPollType,
    ReplyKeyboardMarkup,
    ReplyKeyboardRemove,
    Update,
    InlineKeyboardButton,
    InlineKeyboardMarkup,
)
from telegram.ext import (
    Updater,
    CommandHandler,
    PollAnswerHandler,
    PollHandler,
    MessageHandler,
    CallbackContext,
    CallbackQueryHandler,
)

logging.basicConfig(
    format="%(asctime)s - %(name)s - %(levelname)s - %(message)s", level=logging.INFO
)
logging.getLogger().setLevel(logging.INFO)

# ---------------------------------------------------------------------------------------------------------------------------------------------#

class ChessHandler:
    def __init__(self, stockfish_path, puzzle_path):
      self.user_stockfish = Stockfish(path=stockfish_path)
      self.user_stockfish.set_depth(10)
      self.user_stockfish.set_elo_rating(2000)

      self.cpu_stockfish = Stockfish(path=stockfish_path)
      self.cpu_stockfish.set_depth(7)
      self.cpu_stockfish.set_elo_rating(1300)

      self.puzzle_path = puzzle_path
      self.puzzle_db = pd.read_csv(puzzle_path)


    def get_puzzle(self, rating=None):
        if rating:
          rating_lower = max(rating - 50, self.puzzle_db.min()["Rating"])
          rating_upper = min(rating + 50, self.puzzle_db.max()["Rating"])
          row = self.puzzle_db[(self.puzzle_db.Rating>=rating_lower) & (self.puzzle_db.Rating<=rating_upper)].sample()
        else:
          row = self.puzzle_db.sample()
        FEN = row["FEN"].values[0]
        moves = row["Moves"].values[0]
        rating = row["Rating"].values[0]
        solution_line = moves.split(" ")
        first_move = solution_line.pop(0)
        board = chess.Board(FEN)
        move = chess.Move.from_uci(first_move)
        board.push(move)
        return board, solution_line, rating


    def get_mcq_choices(self, board, solution_san=None, choices_count=4, top_moves_count=7):
        FEN = board.fen()
        self.user_stockfish.set_fen_position(FEN)
        top_moves = self.user_stockfish.get_top_moves(top_moves_count)
        if len(top_moves) == 0:
          return ["Error", "No legal moves found", 0]
        choices = [ChessHandler.uci_to_san(board, top_moves[i]["Move"]) for i, _ in enumerate(top_moves)]

        if not solution_san:
          solution_san = choices[0]
        if choices_count < len(choices):
          choices = random.sample(choices, choices_count)

        if solution_san in choices:
          choices.remove(solution_san)
        else:
          choices.pop()
        solution_ind = random.randint(0,len(choices))
        choices.insert(solution_ind, solution_san)
        if len(choices) == 1:
          choices = choices * 2



        return choices, solution_ind


    def cpu_move(self, board):

        FEN = board.fen()
        self.cpu_stockfish.set_fen_position(FEN)
        cpu_move = self.cpu_stockfish.get_best_move()
        move = chess.Move.from_uci(cpu_move)
        board.push(move)

        return board


    def generate_puzzle(self, rating=None):
        board, solution_line, rating = self.get_puzzle(rating)
        board_img = ChessHandler.get_board_img(board)

        solution_uci = solution_line[0]
        solution_san = ChessHandler.uci_to_san(board, solution_uci)

        choices, solution_ind = self.get_mcq_choices(board, solution_san)

        turn = "White" if board.turn else "Black"
        prompt = f"\U0001F9E9 Chess Puzzle \U0001F9E9\n{turn} to move."
        explanation = "Solution line (in UCI): " + ", ".join(solution_line) + "\n Rating: {}".format(rating)
        return board_img, choices, solution_ind, prompt, explanation


    def new_votechess(self):
        board = chess.Board()
        # Randomize starting player
        if random.choice([True, False]):
          board = self.cpu_move(board)

        board_img = ChessHandler.get_board_img(board)

        turn = "White" if board.turn else "Black"
        prompt = f"{turn} to move"
        choices, solution_ind = self.get_mcq_choices(board, choices_count=5, top_moves_count=8)
        prompt = "\U0001F4CA Vote Chess \U0001F4CA\n" + prompt
        return board_img, choices, solution_ind, prompt, board


    def generate_votechess(self, board, move=None):
        player_turn = board.turn
        if move:
          board.push_san(move) # move must be in san format
          outcome = board.outcome()
          if not outcome:
            board = self.cpu_move(board)

        outcome = board.outcome()
        # Case: Game has ended
        if outcome:
          winner = outcome.winner
          reason = chess.Termination(outcome.termination).name.lower()
          score = outcome.result()
          if winner is None:
            result = "drew"
            text = "That was close!"
          elif winner == player_turn:
            result = "won"
            text = "Congratulations!"
          else:
            result = "lost"
            text = "*Insert generic consolation text here*"
          prompt = f"The game has ended! You {result} {score} by {reason}. {text}\nUse /votechess to start a new game."
          choices, solution_ind = [], -1

        # Case: Game has not ended
        else:
          turn = "White" if board.turn else "Black"
          prompt = f"{turn} to move"
          choices, solution_ind = self.get_mcq_choices(board, choices_count=random.randint(3,4), top_moves_count=5)


        prompt = "\U0001F4CA Vote Chess \U0001F4CA\n" + prompt
        board_img = ChessHandler.get_board_img(board)
        return board_img, choices, solution_ind, prompt, board


    # Static functions
    def uci_to_san(board, uci:str):
        move = chess.Move.from_uci(uci)
        san = board.san(move)
        return san

    def get_board_img(board: chess.Board):
        try:
          last_move = board.peek()
        except:
          last_move = None
        boardsvg = chess.svg.board(board=board,flipped = not board.turn, lastmove = last_move)
        svg2png(bytestring=boardsvg,write_to='board.png')
        board_img = open("board.png", "rb")
        return board_img


# ---------------------------------------------------------------------------------------------------------------------------------------------#


# Telegram utility functions

def is_queued(job_queue, job_name):
    if len(job_queue.get_jobs_by_name(job_name)) == 0:
      return False
    else:
      return True

def remove_queued(job_queue, job_name):
    for job in job_queue.get_jobs_by_name(job_name):
        job.schedule_removal()


# Telegram command handlers
def start(update: Update, context: CallbackContext) -> None:
    """Inform user about what this bot can do"""
    text = """
Beep boop! Welcome to the Chessbot-3000. This bot lets you discover your inner chess with your friends!

Utility
/start - Displays the commands available


\U0001F9E9 Chess Puzzles \U0001F9E9
Challenge puzzles from the Lichess puzzle database!
/puzzle - Sends a puzzle
/schedule_dailypuzzle HHMM - Schedules a puzzle to be sent everyday (UTC)
/stop_dailypuzzle - Clears all daily puzzle schedules


\U0001F4CA Vote Chess \U0001F4CA
Team up with your friends to win Stockfish!
/votechess - Ends the current vote and initiates the next turn
/schedule_votechess HHMM - Schedules a vote chess to be sent everyday (UTC)
/stop_votechess - Clears all vote chess schedules
    """
    reply_keyboard = [["/puzzle", "/votechess"]]
    reply_markup = ReplyKeyboardMarkup(
            reply_keyboard, one_time_keyboard=True, input_field_placeholder="Select command to start."
        )
    context.bot.send_message(chat_id=update.effective_chat.id, text=text, reply_markup=reply_markup)


def schedule_daily_puzzle(update: Update, context: CallbackContext) -> None:
    chat_id = update.effective_chat.id
    job_name = "daily_puzzle_" + str(chat_id)

    if context.args and context.args[0].isdigit():
            time_str = context.args[0]
    else:
        time_str = "1400" # 2pm UTC -> 10pm SGT
    hour = int(time_str[:2])
    minute = int(time_str[2:])
    time = datetime.time(hour=hour, minute=minute, second=random.randint(30,59))

    context.job_queue.run_daily(send_puzzle, time=time, context=chat_id, name=job_name)
    context.bot.send_message(chat_id=chat_id, text=f"Scheduling daily puzzle at {time_str}H (UTC) everyday.")


def stop_daily_puzzle(update: Update, context: CallbackContext) -> None:
    chat_id = update.effective_chat.id
    job_name = "daily_puzzle_" + str(chat_id)

    remove_queued(context.job_queue, job_name)
    context.bot.send_message(chat_id=update.effective_chat.id, text="Daily puzzle schedule has been cleared!")


def puzzle(update: Update, context: CallbackContext) -> None:
    context.job_queue.run_once(send_puzzle, 0.1, context=update.message.chat_id)


def send_puzzle(context: CallbackContext) -> None:
    chat_id = context.job.context
    board_img, choices, solution_ind, prompt, explanation = chess_handler.generate_puzzle()

    context.bot.send_photo(photo=board_img,chat_id=chat_id)

    message = context.bot.send_poll(
        question = prompt, options = choices, correct_option_id=solution_ind,
        type=Poll.QUIZ, allows_multiple_answers = False, explanation=explanation,
        chat_id=chat_id, is_anonymous = False
    )


def schedule_vote_chess(update: Update, context: CallbackContext) -> None:
    chat_id = update.effective_chat.id
    job_name = "vote_chess_" + str(chat_id)

    if context.args and context.args[0].isdigit():
        time_str = context.args[0]
    else:
        time_str = "0200" # 2am UTC -> 10am SGT
    hour = int(time_str[:2])
    minute = int(time_str[2:])
    time = datetime.time(hour=hour, minute=minute, second=random.randint(0,29))

    context.job_queue.run_daily(send_vote_chess, time=time, context=update.message.chat_id, name=job_name)
    context.bot.send_message(chat_id=update.effective_chat.id, text=f"Scheduling vote chess at {time_str}H (UTC) everyday.")


def stop_vote_chess(update: Update, context: CallbackContext):
    chat_id = update.effective_chat.id
    job_name = "vote_chess_" + str(chat_id)
    remove_queued(context.job_queue, job_name)
    vc_data = context.bot_data.get("vote_chess")
    if vc_data and vc_data.get(chat_id):
      vc_data.pop(chat_id)
    context.bot.send_message(chat_id=update.effective_chat.id, text="Vote chess schedule has been cleared!")


def vote_chess(update: Update, context: CallbackContext):
    chat_id = update.effective_chat.id
    context.job_queue.run_once(send_vote_chess, 0.1, context=chat_id)


def send_vote_chess(context: CallbackContext) -> None:
    chat_id = context.job.context
    job_name = "vote_chess_" + str(chat_id)
    vc_data = context.bot_data.get("vote_chess")
    if not vc_data: vc_data = {}

    chat_data = vc_data.get(chat_id)
    # Case: Game has not been initialized
    if not chat_data:
        board_img, choices, solution_ind, prompt, board = chess_handler.new_votechess()

    # Case: Game has been initialized
    else:
      board = chat_data.get("board")
      # Get the most voted move
      moves = chat_data.get("player_moves")
      choices = []
      for player_choice in moves.values():
        choices.extend(player_choice)

      # Case: Nobody voted -> generate poll from the same position
      if len(choices) == 0:
        board_img, choices, solution_ind, prompt, board = chess_handler.generate_votechess(board, None)
      # Case: Top move exists
      else:
        top_choice = max(set(choices), key = choices.count) # If tie, selects first index
        top_move = chat_data.get("move_choices")[top_choice]
        board_img, choices, solution_ind, prompt, board = chess_handler.generate_votechess(board, top_move)

    context.bot.send_photo(photo=board_img,chat_id=chat_id)
    cleaned_choices = [choice.replace("#", "+") for choice in choices]
    # Case: Game has not ended
    if solution_ind >= 0:
      message = context.bot.send_poll(
          question = prompt, options = cleaned_choices, chat_id=chat_id, is_anonymous = False
      )

      data = {
          chat_id: {
              "board": board,
              "current_poll_id": message.poll.id,
              "move_choices": choices,
              "player_moves": {}
          }
      }
      vc_data.update(data)

    # Case: Game has ended
    else:
      context.bot.send_message(chat_id=chat_id, text=prompt)
      vc_data.pop(chat_id)

    context.bot_data.update({"vote_chess": vc_data})





def receive_poll_answer(update: Update, context: CallbackContext) -> None:

    answer = update.poll_answer
    poll_id = answer.poll_id
    user_id = answer["user"]['id']
    option_ids = answer["option_ids"]

    vc_data = context.bot_data.get("vote_chess")
    if vc_data:
      for chat_id in vc_data.keys():
        chat_data = vc_data.get(chat_id)
        current_poll_id = chat_data["current_poll_id"]

        if poll_id == current_poll_id:
          chat_data["player_moves"][user_id] = option_ids

          break
    #print(context.bot_data)


def view(update: Update, context: CallbackContext) -> None:
    data = context.bot_data.get("vote_chess")
    print(context.bot_data)
    print(context.bot_data.get("Random"))
    payload = {"vote_chess": data}
    context.bot_data.update(payload)


def test(update: Update, context: CallbackContext):
    print(context.args)






def main() -> None:
    """Run bot."""
    # Create the Updater and pass it your bot's token.
    updater = Updater(TOKEN)
    dp = updater.dispatcher

    # add handlers
    dp.add_handler(CommandHandler('start', start))

    dp.add_handler(PollAnswerHandler(receive_poll_answer))

    dp.add_handler(CommandHandler('puzzle', puzzle))
    dp.add_handler(CommandHandler('schedule_dailypuzzle', schedule_daily_puzzle))
    dp.add_handler(CommandHandler('stop_dailypuzzle', stop_daily_puzzle))

    dp.add_handler(CommandHandler('votechess', vote_chess))
    dp.add_handler(CommandHandler('schedule_votechess', schedule_vote_chess))
    dp.add_handler(CommandHandler('stop_votechess', stop_vote_chess))

    dp.add_handler(CommandHandler('test', test))

    # Start the Bot
    updater.start_polling()
    logging.info("Chessbot initialized.")

    # Run the bot until the user presses Ctrl-C or the process receives SIGINT,
    # SIGTERM or SIGABRT

    updater.idle()
    logging.warning("Ending chessbot process.")

TOKEN = "6608467118:AAEyltpXJbkFTbn88XePgWEZOhrm7KcaGwU"
stockfish_path = "./stockfish/stockfish-ubuntu-x86-64-avx2"
puzzle_path = "./chess_puzzles.csv"
chess_handler = ChessHandler(stockfish_path, puzzle_path)

main()

2023-09-22 03:09:07,995 - apscheduler.scheduler - INFO - Scheduler started
2023-09-22 03:09:09,981 - root - INFO - Chessbot initialized.
2023-09-22 03:10:54,511 - telegram.ext.updater - INFO - Received signal 2 (SIGINT), stopping...
2023-09-22 03:10:54,512 - apscheduler.scheduler - INFO - Scheduler has been shut down
2023-09-22 03:10:58,463 - root - WARNING - Ending chessbot process.


In [10]:
import os
os.getcwd()

'/workspaces/codespaces-jupyter'

### Misc

In [ ]:
import random
stockfish = Stockfish(path="/content/stockfish/stockfish-ubuntu-x86-64-avx2")
# Set stockfish configurations
stockfish.set_depth(10)
stockfish.set_elo_rating(2000)
board = chess.Board("r6k/pp2r2p/4Rp1Q/3p4/8/1N1P2R1/PqP2bPP/7K b - - 0 24")
# Instruct stockfish to generate k top moves
solution_uci = "b2b1"
solution_san = uci_to_san(board, solution_uci)


FEN = board.board_fen()
stockfish.set_fen_position(FEN)
quiz_answers_count = 4
top_moves = stockfish.get_top_moves(7)
answers = [uci_to_san(board, top_moves[i]["Move"]) for i, _ in enumerate(top_moves)]
answers = random.sample(answers, quiz_answers_count)
if solution_san in answers:
  answers.remove(solution_san)
else:
  answers.pop()
solution_ind = random.randint(0,quiz_answers_count)
answers.insert(solution_ind, solution_san)
display(board)
print(answers, solution_san)

In [ ]:
def generate_inline_chessboard(board):
    files = ["a", "b", "c", "d", "e", "f", "g", "h"]
    ranks = [str(i) for i in range(8,0,-1)]
    indices = ['♚','♛','♜','♝','♞','♟','⭘','♙','♘','♗','♖','♕','♔']
    new_indices = ['♚','♛','♜','♝','♞','♟',' ','♙','♘','♗','♖','♕','♔']
    new_indices = ['♔','♕','♖','♗','♘','♟',' ','♙','♞','♝','♜','♛','♚']

    unicode_board = [
        [new_indices[indices.index(c)] for c in row.split()]
        for row in board.unicode(invert_color=False).split('\n')
    ]


    inline_chessboard = [
        [InlineKeyboardButton(unicode_board[i][j], callback_data=f"{f}{r}") for j,f in enumerate(files)] for i,r in enumerate(ranks)
    ]

    if board.turn == chess.BLACK:
        for row in inline_chessboard:
          row.reverse()
        inline_chessboard.reverse()

    reply_markup = InlineKeyboardMarkup(inline_chessboard)
    return reply_markup
def button(update: Update, context: CallbackContext) -> None:
    """Parses the CallbackQuery and updates the message text."""
    query = update.callback_query

    # CallbackQueries need to be answered, even if no notification to the user is needed
    # Some clients may have trouble otherwise. See https://core.telegram.org/bots/api#callbackquery
    query.answer()

    context.bot.send_message(chat_id=update.effective_chat.id, text=f"{query.data}")

    dp.add_handler(CallbackQueryHandler(button))
def check_answers(update: Update, context: CallbackContext):
    text = update.message.text.strip()
    chat_id = update.effective_chat.id
    solutions = context.bot_data[chat_id]["solutions"]
    for solution in solutions.values():
        if text in solution:
            context.bot.send_message(chat_id=update.effective_chat.id, text=f"Good job! The solution was {solutions[1]}.")
            puzzle(update, context)
            break

    dp.add_handler(MessageHandler(Filters.text & (~Filters.command), check_answers))